In [ ]:
# Some jupyter notebook magic to reload modules automatically when they change
# not necessary for this specific notebook but useful in general
%load_ext autoreload
%autoreload 2

In [ ]:
from definitions import ROOT_DIR
import os

# The PSDM specific input models can be found in the pypsdm.models.input submodules
from pypsdm.models.input import GridContainer

# but can also be imported from the root module
from pypsdm import GridContainer


grid_path = os.path.join(ROOT_DIR, "tests", "resources", "simple_grid", "input")
# IO data models in general have a from_csv method to parse psdm files
grid = GridContainer.from_csv(grid_path)
# You can also load a `GridWithResults` container which additionally contains the result
# data. For more details see the `result_models.ipynb` notebook

In [ ]:
from pypsdm.plots.grid import grid_plot

# Use the grid_plot method to visualize the grid model
# only works if the underlying node input files have associated coordinates
grid_plot(grid)

In [ ]:
# You can get a graph representation of the grid
graph = grid.raw_grid.build_networkx_graph()
# And a list of all the branches in the grid
branches_list = grid.raw_grid.get_branches()
branches_subgraphs = grid.raw_grid.get_branches(as_graphs=True)
branches_list, branches_subgraphs

In [ ]:
# A grid container consists of a raw grid container
raw_grid = grid.raw_grid
# consisting of lines, transformers and so on
lines = raw_grid.lines
# and a participants container
participants = grid.participants
# consisting of loads, pv plants
pvs = participants.pvs

# You can also directly access the nested objects from the container classes
pvs = grid.pvs

# The base data structure of all input model is a pandas DataFrame accessible via .data
pvs.data

In [ ]:
# You can access the columns via the data frame
pvs.data["s_rated"]

In [ ]:
# or directly via the property attribute of the class
pvs.s_rated

In [ ]:
# The respective classes implement some useful methods for dealing with the data
# e.g. retrieve all elements connected to one or multiple specified nodes
# (Please check out the classes to see all the implemented methods)
nodal_participants = participants.filter_by_nodes(grid.nodes.uuid.to_list()[::2])
nodal_participants.pvs.data

In [ ]:
from tempfile import TemporaryDirectory

# You can write data models with their respective to_csv method
with TemporaryDirectory() as temp_dir:
    grid.to_csv(temp_dir)
    grid_from_csv = GridContainer.from_csv(temp_dir)

    grid.compare(grid_from_csv)
    assert grid.raw_grid.nodes == grid_from_csv.raw_grid.nodes
    assert grid.loads == grid_from_csv.loads
    # the == operator is overloaded to compare the underlying dataframes
    assert grid == grid_from_csv